* 通常在Python中我们进行并发编程一般都是使用多线程或者多进程来实现的，对于CPU计算密集型任务由于GIL的存在通常使用多进程来实现，而对于IO密集型任务可以通过线程调度来让线程在执行IO任务时让出GIL，从而实现表面上的并发。
* 其实对于IO密集型任务我们还有一种选择就是协程。协程，又称微线程，英文名Coroutine，是运行在单线程中的“并发”，协程相比多线程的一大优势就是省去了多线程之间的切换开销，获得了更高的运行效率。Python中的异步IO模块asyncio就是基本的协程模块。

### 1. 协程
* 协程的切换不同于线程切换，是由程序自身控制的，没有切换的开销。协程不需要多线程的锁机制，因为都是在同一个线程中运行，所以没有同时访问数据的问题，执行效率比多线程高很多。
* 因为协程是单线程执行，那怎么利用多核CPU呢？最简单的方法是多进程+协程，既充分利用多核，又充分发挥协程的高效率，可获得极高的性能。
* 进程/线程：操作系统提供的一种并发处理任务的能力。
* 协程：程序员通过高超的代码能力，在代码执行流程中人为的实现多任务并发，是单个线程内的任务调度技巧。
* 多进程和多线程体现的是操作系统的能力，而协程体现的是程序员的流程控制能力。看下面的例子，甲，乙两个工人模拟两个工作任务交替进行，在单线程内实现了类似多线程的功能

In [1]:
import time

def task1():
    while True:
        yield "<甲>也累了，让<乙>工作一会儿"
        time.sleep(1)
        print("<甲>工作了一段时间.....")


def task2(t):
    next(t)
    while True:
        print("-----------------------------------")
        print("<乙>工作了一段时间.....")
        time.sleep(2)
        print("<乙>累了，让<甲>工作一会儿....")
        ret = t.send(None)
        print(ret)
    t.close()

if __name__ == '__main__':
    t = task1()
    task2(t)

-----------------------------------
<乙>工作了一段时间.....
<乙>累了，让<甲>工作一会儿....
<甲>工作了一段时间.....
<甲>也累了，让<乙>工作一会儿
-----------------------------------
<乙>工作了一段时间.....
<乙>累了，让<甲>工作一会儿....
<甲>工作了一段时间.....
<甲>也累了，让<乙>工作一会儿
-----------------------------------
<乙>工作了一段时间.....
<乙>累了，让<甲>工作一会儿....


KeyboardInterrupt: 

### 2.yield
* 最早的时候，Python提供了yield关键字，用于制造生成器。也就是说，包含有yield的函数，都是一个生成器
* yield的语法规则是：在yield这里暂停函数的执行，并返回yield后面表达式的值（默认为None），直到被next()方法再次调用时，从上次暂停的yield代码处继续往下执行。当没有可以继续next()的时候，抛出异常，该异常可被for循环处理。

In [2]:
def fib(n):
    a, b = 0, 1
    i = 0
    while i < n:
        yield b
        a, b = b, a+b
        i += 1

if __name__ == '__main__':
    f = fib(10)
    for item in f:
        print(item)

1
1
2
3
5
8
13
21
34
55


### 3. send()
* 最初的yield只能返回并暂停函数，并不能实现协程的功能。后来，Python为它定义了新的功能——接收外部发来的值，这样一个生成器就变成了协程。
* 每个生成器都可以执行send()方法，为生成器内部的yield语句发送数据。此时yield语句不再只是yield xxxx的形式，还可以是var = yield xxxx的赋值形式。它同时具备两个功能，一是暂停并返回函数，二是接收外部send()方法发送过来的值，重新激活函数，并将这个值赋值给var变量！

In [3]:
def simple_coroutine():
    print('-> 启动协程')
    y = 10
    x = yield y
    print('-> 协程接收到了x的值:', x)

my_coro = simple_coroutine()
ret = next(my_coro)
print(ret)
my_coro.send(10)

-> 启动协程
10
-> 协程接收到了x的值: 10


StopIteration: 

### 4. @asyncio.coroutine与yield from
* @asyncio.coroutine：asyncio模块中的装饰器，用于将一个生成器声明为协程。
* yield from 其实就是等待另外一个协程的返回。
* 下面这段代码，我们创造了一个协程display_date(num, loop)，然后它使用关键字yield from来等待协程asyncio.sleep(2)()的返回结果。而在这等待的2s之间它会让出CPU的执行权，直到asyncio.sleep(2)返回结果。asyncio.sleep(2)模拟的其实就是一个耗时2秒的IO读写操作。

In [4]:
import asyncio
import datetime

@asyncio.coroutine  # 声明一个协程
def display_date(num, loop):
    end_time = loop.time() + 10.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        yield from asyncio.sleep(2)  # 阻塞直到协程sleep(2)返回结果
loop = asyncio.get_event_loop()  # 获取一个event_loop
tasks = [display_date(1, loop), display_date(2, loop)]
loop.run_until_complete(asyncio.gather(*tasks))  # "阻塞"直到所有的tasks完成
loop.close()

Loop: 2 Time: 2018-11-22 09:21:44.265471
Loop: 1 Time: 2018-11-22 09:21:44.265695
Loop: 2 Time: 2018-11-22 09:21:46.268356
Loop: 1 Time: 2018-11-22 09:21:46.269303
Loop: 2 Time: 2018-11-22 09:21:48.272431
Loop: 1 Time: 2018-11-22 09:21:48.274034
Loop: 2 Time: 2018-11-22 09:21:50.276245
Loop: 1 Time: 2018-11-22 09:21:50.276766
Loop: 2 Time: 2018-11-22 09:21:52.279649
Loop: 1 Time: 2018-11-22 09:21:52.280147
Loop: 2 Time: 2018-11-22 09:21:54.282648
Loop: 1 Time: 2018-11-22 09:21:54.282852


### 5. async和await
* Python3.5中对协程提供了更直接的支持，引入了async/await关键字


In [1]:
import asyncio
import datetime

async def display_date(num, loop):      # 注意这一行的写法
    end_time = loop.time() + 10.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(2)  # 阻塞直到协程sleep(2)返回结果

loop = asyncio.get_event_loop()  # 获取一个event_loop
tasks = [display_date(1, loop), display_date(2, loop)]
loop.run_until_complete(asyncio.gather(*tasks))  # "阻塞"直到所有的tasks完成
loop.close()

Loop: 1 Time: 2018-11-22 09:24:14.451968
Loop: 2 Time: 2018-11-22 09:24:14.452192
Loop: 1 Time: 2018-11-22 09:24:16.454954
Loop: 2 Time: 2018-11-22 09:24:16.455803
Loop: 1 Time: 2018-11-22 09:24:18.457261
Loop: 2 Time: 2018-11-22 09:24:18.457630
Loop: 1 Time: 2018-11-22 09:24:20.460330
Loop: 2 Time: 2018-11-22 09:24:20.461131
Loop: 1 Time: 2018-11-22 09:24:22.464021
Loop: 2 Time: 2018-11-22 09:24:22.464533
Loop: 1 Time: 2018-11-22 09:24:24.467380
Loop: 2 Time: 2018-11-22 09:24:24.469023


### 6. asyncio模块
* asyncio的使用可分三步走：
    * 创建事件循环
    * 指定循环模式并运行
    * 关闭循环
* 通常我们使用asyncio.get_event_loop()方法创建一个循环
* 运行循环有两种方法：一是调用run_until_complete()方法，二是调用run_forever()方法。run_until_complete()内置add_done_callback回调函数，run_forever()则可以自定义add_done_callback()

In [1]:
import asyncio

async def func(future):
    await asyncio.sleep(1)
    future.set_result('Future is done!')

if __name__ == '__main__':

    loop = asyncio.get_event_loop()
    future = asyncio.Future()
    asyncio.ensure_future(func(future))
    print(loop.is_running())   # 查看当前状态时循环是否已经启动
    loop.run_until_complete(future)   # 使用run_until_complete()方法
    print(future.result())
    loop.close()

False
Future is done!


In [1]:
import asyncio

async def func(future):
    await asyncio.sleep(1)
    future.set_result('Future is done!')

# 自定义回调函数
def call_result(future):
    print(future.result())
    loop.stop()

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    future = asyncio.Future()
    asyncio.ensure_future(func(future))
    future.add_done_callback(call_result)        # 注意这行,自定义回调函数
    try:
        loop.run_forever()   # 使用run_forever()方法
    finally:
        loop.close()

Future is done!
